# Знакомство с данными и агрегация

In [55]:
import pandas as pd
import calendar
from datetime import date, datetime, timedelta


В каждом из файлов содержится следующая информация о поездках:

| Code | Value   |
|------|------|
|tpep_pickup_datetime | время начала поездки|
|tpep_dropoff_datetime | время окончания поездки|
|pickup_longitude pickup_latitude | долгота и широта точки начала поездки|
|dropoff_longitude dropoff_latitude | долгота и широта точки окончания поездки|
|passenger_count | количество пассажиров|
|trip_distance | расстояние по счётчику|
|RatecodeID | тип тарифа (одна из шести категорий)|
|payment_type | способ оплаты (одна из шести категорий)|
|fare_amount | стоимость поездки по счётчику|
|extra | налог на счётчик|
|mta_tax | доплата за поездки в пиковые часы и ночью|
|tolls_amount | доплата за проезд по платным дорогам|
|improvement_surcharge | доплата за проезд, взимаемая с каждой поездки с января 2015|
|tip_amount | размер чаевых|
|total_amount | общая стоимость поездки|
|VendorID | провайдер данных (одна из двух категорий)|
|store_and_fwd_flag | бинарный флаг, показывающий, были ли данные о поезде получены немедленно после её окончания, или какое-то время хранились в памяти автомобиля.|



Загрузите данные за май 2016 года.

In [2]:
data=pd.read_csv('D:/Загрузки/yellow_tripdata_2016-05.csv')
print (data.shape)
data.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,2016-05-01 00:00:00,2016-05-01 00:17:31,1,3.60,-73.985901,40.768040,1,N,-73.983986,40.730099,1,15.0,0.5,0.5,1.50,0.0,0.3,17.80
1,2,2016-05-01 00:00:00,2016-05-01 00:07:31,1,1.68,-73.991577,40.744751,1,N,-73.975700,40.765469,1,7.5,0.5,0.5,0.88,0.0,0.3,9.68
2,2,2016-05-01 00:00:00,2016-05-01 00:07:01,6,1.09,-73.993073,40.741573,1,N,-73.980995,40.744633,1,6.5,0.5,0.5,1.56,0.0,0.3,9.36
3,2,2016-05-01 00:00:00,2016-05-01 00:19:47,1,4.21,-73.991943,40.684601,1,N,-74.002258,40.733002,1,17.0,0.5,0.5,3.66,0.0,0.3,21.96
4,2,2016-05-01 00:00:00,2016-05-01 00:06:39,1,0.56,-74.005280,40.740192,1,N,-73.997498,40.737564,1,6.0,0.5,0.5,1.46,0.0,0.3,8.76


## Проведим фильтрацию данных

In [28]:
def filter_trips(data):
    # Оставляем поездки с ненулевой длительностью
    fil=((data.tpep_dropoff_datetime>data.tpep_pickup_datetime)&
    # Оставляем поездки с ненулевым количеством пассажиров
    (data.passenger_count>0)& 
    # Оставляем поездки с ненулевым расстоянием поездки по счётчику
    (data.trip_distance>0)& 
    # Оставляем поездки с координатами начала, попадающими в прямоугольник Нью-Йорка
    #  от -74.25559 до -73.70001 градусов долготы
    # от 40.49612 до 40.91553 широты.
    (data.pickup_longitude>=-74.25559)&(data.pickup_longitude<=-73.70001)&(data.pickup_latitude>=40.49612)&(data.pickup_latitude<=40.91553))
    return data[fil]

In [29]:
data=filter_trips(data)

In [30]:
data.shape

(11626521, 19)

## Агрегируем данные за месяц по часам и областям 

In [65]:
# Преобразовываем тип колонок к типу дата
data.tpep_pickup_datetime= pd.to_datetime(data.tpep_pickup_datetime)
data.tpep_dropoff_datetime= pd.to_datetime(data.tpep_dropoff_datetime)

In [83]:
# Строим новую колонку - час посадки
data['z_tpep_pickup_hour']=data.tpep_pickup_datetime.apply (lambda x:x.hour)

In [123]:
regions=pd.read_csv('regions.csv',sep=';')
regions.head(100)

,region,west,east,south,north
0,1,-74.255590,-74.244478,40.496120,40.504508
1,2,-74.255590,-74.244478,40.504508,40.512896
2,3,-74.255590,-74.244478,40.512896,40.521285
3,4,-74.255590,-74.244478,40.521285,40.529673
4,5,-74.255590,-74.244478,40.529673,40.538061
5,6,-74.255590,-74.244478,40.538061,40.546449
6,7,-74.255590,-74.244478,40.546449,40.554837
7,8,-74.255590,-74.244478,40.554837,40.563226
8,9,-74.255590,-74.244478,40.563226,40.571614
9,10,-74.255590,-74.244478,40.571614,40.580002


In [146]:
for index, region in regions.iterrows():
    print (index)
    flt=((region.west<=data.pickup_longitude)&(region.east>=data.pickup_longitude)
    &(region.south<=data.pickup_latitude)&(region.north>=data.pickup_latitude))
    data.loc[flt,'z_region']=index


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

KeyboardInterrupt: 

In [134]:
data[flt]

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,...,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,z_tpep_pickup_hour,z_region
4892188,1,2016-05-13 15:08:02,2016-05-13 15:40:57,1,1.8,-74.241211,40.767982,1,N,-74.241211,...,2,19.5,0.0,0.5,0.00,0.0,0.3,20.30,15,82.0
4902539,1,2016-05-13 15:46:33,2016-05-13 16:57:31,2,16.6,-74.241211,40.767982,3,N,-74.177826,...,2,76.5,0.0,0.0,0.00,12.5,0.3,89.30,15,82.0
6112421,1,2016-05-16 13:08:34,2016-05-16 13:20:19,1,1.7,-74.241478,40.767826,1,N,-73.976280,...,1,9.0,0.0,0.5,1.00,0.0,0.3,10.80,13,82.0
9536990,1,2016-05-22 13:24:53,2016-05-22 13:46:21,1,4.3,-74.244110,40.765919,1,N,-73.998039,...,1,18.5,0.0,0.5,3.86,0.0,0.3,23.16,13,82.0
